In [ ]:
import os
from cv2 import cv2
from PIL import Image
from paddleocr import PaddleOCR, draw_ocr
 
 
# 读入视频
video_filename = r'F:\pycharm_project\OCR\video\demo.mp4'
videoCap = cv2.VideoCapture(video_filename)
# 切换图片保存目录
os.chdir(r'F:\pycharm_project\OCR\pics')
 
np_pic_list = []
for i in range(3, 10):
    i = i * 700  # i代表随便抽取的某些帧
    videoCap.set(cv2.CAP_PROP_POS_FRAMES, i)  # 设置要获取的帧号
    TorF, frame = videoCap.read()  # read方法返回一个布尔值和一个视频帧
    # im = frame[:, :, 0] #  显示全图
    # 确定字幕的范围，这里仅针对该视频作展示实际情况需要更改
    np_img = frame[870:955, :]
    # 实现array到image的转换
    # oir_pic = Image.fromarray(np_img)
    # 这里颜色会出现改变，问题应该是RGB与BGR编码格式变化引起的
    # oir_pic.save('pic_' + str(i) + '.jpg')
    # 将字幕段进行颜色处理，未达到阈值的颜色将会变为白色背景
    retVal, bw_img = cv2.threshold(np_img, 245, 245, cv2.THRESH_BINARY_INV)
    # 逐段追加
    np_pic_list.append(bw_img)
 
# 垂直拼接并保存成一张图
Image.fromarray(cv2.vconcat(np_pic_list)).save('result_1.jpg')
 
# 调用paddleOCR接口
ocr = PaddleOCR(use_angle_cls=True, lang="ch")
img_path = r'F:\pycharm_project\OCR\pics\result_1.jpg'
result = ocr.ocr(img_path, cls=True)
for line in result:
    print(line)
 
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
 
im_show = draw_ocr(image, boxes, txts, scores, font_path='/path/to/PaddleOCR/doc/simfang.ttf')
im_show = Image.fromarray(im_show)
im_show.save('result_2.jpg')
